Faster R-CNN

Import  (tutorial: https://github.com/tensorflow/models/blob/master/object_detection/object_detection_tutorial.ipynb)

In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import cv2

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

from densenet import densenet_model


Using TensorFlow backend.


In [2]:
# This is needed to display the images.
%matplotlib inline
sys.path.append(os.path.abspath('./tf_models'))
sys.path.append(os.path.abspath('./tf_models/slim'))

In [3]:
#READ finetuned FASTER_RCNN model
# Path to frozen detection graph. This is the actual model that is used for the object detection.
obj_detection_graph_path = 'models/faster_rcnn/output/frozen_inference_graph.pb'
# List of the strings that is used to add correct label for each box.
labels = "data/label_map.pbtxt"
num_classes = 2

detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(obj_detection_graph_path, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
    
label_map = label_map_util.load_labelmap(labels)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [49]:
#READ finetuned DENSENET MODEL
densenet_img_rows, densenet_img_cols = 224, 224  # Resolution of inputs
channel = 3
classification_model_path = 'models/densenet/densenet_finetuned.h5'
classification_model = densenet_model(img_rows=densenet_img_rows, img_cols=densenet_img_cols,
                                        color_type=channel, num_classes=num_classes, weights_path=classification_model_path)


/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:182: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (7, 7), name="conv1", use_bias=False, strides=(2, 2))`
  x = Convolution2D(nb_filter, 7, 7, subsample=(2, 2), name='conv1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv2_1_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv2_1_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:330: UserWarning: The `merge` function is deprecated and will be removed after

/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv3_8_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv3_9_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv3_9_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="c

/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv4_11_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv4_12_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv4_12_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name

/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv4_26_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv4_27_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv4_27_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name

/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv5_5_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv5_5_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv5_6_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), nam

/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv5_20_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), name="conv5_20_x2", use_bias=False)`
  x = Convolution2D(nb_filter, 3, 3, name=conv_name_base + '_x2', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:263: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (1, 1), name="conv5_21_x1", use_bias=False)`
  x = Convolution2D(inter_channel, 1, 1, name=conv_name_base + '_x1', bias=False)(x)
/Users/tuncer/Documents/Dev/machine-learning/projects/capstone/densenet.py:273: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), 

In [54]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def get_obj_img(img,boxes, scores,classes):
    #get the index of the object with highest score
    i = scores.argsort()[10:]
    print (i)
    #print('class %s:%s\nboxes %s\n'%(classes[i], scores[i],boxes[i]))
    box = boxes[i] #box [y_min, x_min, y_max, x_max]
    cropped=img[box[0]:box[2],box[1]:box[3]] #img[y: y + h, x: x + w]
    return (box,cropped)

def transform_img_for_densenet(image, image_size, boxes, img_rows, img_cols):
    #box [y_min, x_min, y_max, x_max]  normalized values
    
    
    (im_width, im_height) = image_size
    images = list()
    for box in boxes[0][:2]:
        print("%s %s %s %s"%(box[0], box[2],box[1],box[3]))
        #normalize
        x_min = int(box[1]*im_width)
        x_max = int(box[3]*im_width)
        y_min = int(box[0]*im_height)
        y_max = int(box[2]*im_height)
        print("%s %s %s %s"%(x_min, x_max, y_min, y_max))
        obj = image[y_min:y_max, x_min:x_max] #img[y: y + h, x: x + w]
        img = cv2.resize(obj, (img_rows, img_cols)).astype(np.float32)
        print (img[0][0][0])
        img[:, :, 0] -= 103.939
        img[:, :, 1] -= 116.779
        img[:, :, 2] -= 123.68
        images.append(img)
    
    print(len(images))
    return np.array(images)

In [6]:
video_img_paths = ['data/videos/frames/297.png' ]

# Size, in inches, of the output images.
IMAGE_SIZE = (300, 300)

In [56]:

#cap = cv2.VideoCapture('data/videos/project_video.mp4')
        
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        #while(cap.isOpened()):
            
            #ret, frame = cap.read()
            #frame = cv2.resize(cv2.imread('data/frames/580.png'),(img_rows, img_cols)).astype(np.float32)
            image = Image.open('data/frames/580.png')
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            image_np = load_image_into_numpy_array(image)

            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]           
            image_np_expanded = np.expand_dims(image_np, axis=0)
            
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            print(1)
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
              [boxes, scores, classes, num_detections],
              feed_dict={image_tensor: image_np_expanded})

1


In [63]:

#print(2)
#print("boxes\n%s\n\n scores\n%s\n\n classes:%s\n\n\n num_detections:%s\n"%(boxes, scores, classes, num_detections))
# Visualization of the results of a detection.
#box [y_min, x_min, y_max, x_max]
#for box in boxes[0][:1]:
images=transform_img_for_densenet(image_np, image.size, boxes, densenet_img_rows, densenet_img_cols)
#densenet_input = np.array([img])
print('\n\n\n\nPredicting')
predictions = classification_model.predict(images, batch_size=1, verbose=1)
print(predictions)
#[<confidence on obj is non-vehicle>,<confidence score on object is vehicle>]
classes = np.array([1, 1])
vis_util.visualize_boxes_and_labels_on_image_array(image_np,boxes[0][:2],classes,classes,category_index,use_normalized_coordinates=True,line_thickness=8)
plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image_np)
cv2.imshow('object detection',image_np)

#easy way to draw border
#https://stackoverflow.com/questions/11142851/adding-borders-to-an-image-using-python
    

0.568552 0.650767 0.690838 0.777348
884 995 409 468
0.599456 0.629305 0.262229 0.293844
335 376 431 453




Predicting
2/2 [==============================] - 4s     
[[ 0.29482874  0.70517129]
 [ 0.55600363  0.44399634]]


In [36]:
image.size

(1280, 720)

In [ ]:
file_writer = tf.summary.FileWriter('logs', detection_graph)

In [ ]:
            
            vis_util.visualize_boxes_and_labels_on_image_array(
              image_np,
              np.squeeze(boxes),
              np.squeeze(classes).astype(np.int32),
              np.squeeze(scores),
              category_index,
              use_normalized_coordinates=True,
              line_thickness=8)
            cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break

          plt.figure(figsize=IMAGE_SIZE)
          plt.imshow(image_np)